In [1]:
%cd /home/elijah/video_membership


In [2]:
from app import db
from app.playlist.models import Playlist 
from app.users.models import User
from app.videos.models import Video
from cassandra.cqlengine.management import sync_table

In [3]:
from app.indexing.client import get_index, update_index, update_index
from app.indexing.schemas import *

In [4]:
db.get_session()
sync_table(User)
sync_table(Video)
sync_table(Playlist)

In [5]:
import uuid 
import json
from pydantic import BaseModel , Field, validator, root_validator
from typing import Optional

In [6]:
video_obj = dict(Video.objects.first())
video_obj

In [7]:
class VideoIndexSchema2(BaseModel):
    objectID: str = Field(alias='host_id')
    objectType: str = "Video"
    title: Optional[str]
    path: str = Field(alias='host_id')
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        host_id = v
        return f"/videos/{host_id}"

In [8]:
VideoIndexSchema(**video_obj).dict()

In [9]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

In [10]:
class PlaylistIndexSchema2(BaseModel):
    objectID: uuid.UUID = Field(alias='db_id')
    objectType: str = "Playlist"
    title: Optional[str]
    path: str = Field(default='/')
    
    @root_validator
    def set_default(cls, values):
        objectID = values.get('objectID')
        values['objectID'] = str(objectID)
        values['path'] = f"/playlists/{objectID}"
        return values

In [11]:
##json.loads(PlaylistIndexSchema(**playlist_obj).json())

In [12]:
PlaylistIndexSchema(**playlist_obj).dict()

In [13]:
video_q = [dict(x) for x in Video.objects.all()]
videos_dataset = [VideoIndexSchema(**x).dict() for x in video_q]
videos_dataset

In [14]:
playlist_q = [dict(x) for x in Playlist.objects.all()]
playlist_dataset = [PlaylistIndexSchema(**x).dict() for x in playlist_q]
playlist_dataset

In [15]:
dataset = playlist_dataset + videos_dataset

In [16]:
dataset

In [17]:
index = get_index()

In [18]:
#dir(index)

In [23]:
#idx_response = index.save_objects(dataset).wait()
#len(idx_response[0]['objectIDs'])
updated_count = update_index()
updated_count

In [22]:
index.search("Ranch")